This notebook present the steps to generatea merged catalog files. The reason that we need to generate such files is that:

1. Bliss needs input parameters that contains psf, fluxes, location, source type, galaxy related parameters, but the truth catalog in DC2 does not provide galaxy parameters or psf
2. To get those parameter, we need to use CosmoDC2 dataset and truth-match table, where the galaxy parameters  and psf arestores.
3. To save the time for multiple merging when loading the data, we store the merged dataset into corresponding merged_catalog_{}.pkl files


### Load Catalog

Use [GCRCatalogs](https://data.lsstdesc.org/doc/install_gcr) package to load the dc2 truth catalog and CosomoDC2.




In [1]:
import GCRCatalogs
GCRCatalogs.set_root_dir("/nfs/turbo/lsa-regier/lsstdesc-public/dc2")
truth_cat = GCRCatalogs.load_catalog('desc_dc2_run2.2i_dr6_truth') 
data = truth_cat.get_quantities([
    "id", "match_objectId", "cosmodc2_id", "ra", "dec", "truth_type", 
    "flux_g", "flux_i", "flux_r", "flux_u", "flux_y", "flux_z"
])

In [2]:
config_overwrite = dict(
    catalog_root_dir='/nfs/turbo/lsa-regier/lsstdesc-public/dc2/cosmoDC2'
)
# take long time to load (around 10 mins for 59 files)
galaxy_cat = GCRCatalogs.load_catalog('desc_cosmodc2', config_overwrite)
galaxy = galaxy_cat.get_quantities([
    "galaxy_id", "position_angle_true", "size_minor_disk_true", 
    "size_disk_true", "size_minor_bulge_true", 
    "size_bulge_true", "bulge_to_total_ratio_i"
])

In [3]:
import GCRCatalogs
match_cat = GCRCatalogs.load_catalog('desc_dc2_run2.2i_dr6_object_with_truth_match')
psf_params = match_cat.get_quantities([
    "objectId", "IxxPSF_pixel_g", "IxxPSF_pixel_z", 
    "IxxPSF_pixel_r", "IxxPSF_pixel_i", "IxxPSF_pixel_u", 
    "IxxPSF_pixel_y", "IyyPSF_pixel_g", "IyyPSF_pixel_z", 
    "IyyPSF_pixel_r", "IyyPSF_pixel_i", "IyyPSF_pixel_u", 
    "IyyPSF_pixel_y", "IxyPSF_pixel_g", "IxyPSF_pixel_z", 
    "IxyPSF_pixel_r", "IxyPSF_pixel_i", "IxyPSF_pixel_u", 
    "IxyPSF_pixel_y", "psf_fwhm_g", "psf_fwhm_z", "psf_fwhm_r",
    "psf_fwhm_i", "psf_fwhm_u", "psf_fwhm_y"
])

### Merge Catalog 

In [4]:
import pandas as pd
df_data = pd.DataFrame(data)
df_galaxy = pd.DataFrame(galaxy)
df_psf = pd.DataFrame(psf_params)
merge_data = df_data.merge(
    df_galaxy, 
    left_on = "cosmodc2_id", 
    right_on = "galaxy_id", 
    how = "left" 
)
merge_data_fill = merge_data.fillna(-1)
data_psf = merge_data_fill.merge(
    df_psf, 
    left_on = "match_objectId", 
    right_on = "objectId", 
    how = "left" 
)

# filter the object by flix
bright_merge = data_psf[data_psf['flux_r'] > 50]


### Save to File

In [5]:
import pickle

# create a pickle file 
f = bright_merge.to_pickle("merged_catalog_psf_50.pkl")